In [ ]:
# app.py
import streamlit as st
import sqlite3

# Connect to the database (or create if not exist)
conn = sqlite3.connect('users.db', check_same_thread=False)
c = conn.cursor()

# Tables
c.execute('''CREATE TABLE IF NOT EXISTS users (
    username TEXT PRIMARY KEY,
    password TEXT
)''')
c.execute('''CREATE TABLE IF NOT EXISTS responses (
    username TEXT,
    gender TEXT,
    age TEXT,
    q1 TEXT,
    q2 TEXT,
    q3 TEXT,
    q4 TEXT,
    q5 TEXT,
    q6 TEXT,
    q7 TEXT,
    q8 TEXT
)''')
conn.commit()

# Signup page
def signup():
    st.subheader("🔐 تسجيل حساب جديد")
    username = st.text_input("اسم المستخدم")
    password = st.text_input("كلمة المرور", type="password")
    if st.button("تسجيل"):
        try:
            c.execute("INSERT INTO users VALUES (?, ?)", (username, password))
            conn.commit()
            st.success("تم التسجيل! يمكنك الآن تسجيل الدخول.")
        except:
            st.error("اسم المستخدم مستخدم بالفعل.")

# Login page
def login():
    st.subheader("🔑 تسجيل الدخول")
    username = st.text_input("اسم المستخدم")
    password = st.text_input("كلمة المرور", type="password")
    if st.button("دخول"):
        c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))
        result = c.fetchone()
        if result:
            st.session_state['user'] = username
            st.success("مرحباً بك، تم تسجيل الدخول.")
        else:
            st.error("بيانات الدخول غير صحيحة.")

# Questionnaire page
def questionnaire():
    st.subheader("📝 التقييم النفسي")
    gender = st.radio("ما هو جنسك؟", ["ذكر", "أنثى"])
    age = st.radio("ما هي فئتك العمرية؟", ["18-29", "30-39", "40-49", "50+"])

    q1 = st.text_area("س1: هل تجد نفسك تعاني من التفكير المفرط أو القلق الزائد؟")
    q2 = st.text_area("س2: هل تواجه صعوبة في السيطرة على أفكارك القلقة؟")
    q3 = st.text_area("س3: هل يترافق التفكير المفرط مع أعراض مثل التعب أو قلة النوم؟")
    q4 = st.text_area("س4: هل مررت بفترة اكتئاب لأكثر من أسبوعين؟")
    q5 = st.text_area("س5: هل سببت لك الأعراض ضيقًا نفسيًا كبيرًا؟")
    q6 = st.text_area("س6: هل كانت الأعراض نتيجة أدوية أو مواد؟")
    q7 = st.text_area("س7: (اختياري) أي تفاصيل إضافية؟")
    q8 = st.text_area("س8: (اختياري) متى بدأت هذه الأعراض؟")

    if st.button("إرسال التقييم"):
        c.execute("INSERT INTO responses VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                  (st.session_state['user'], gender, age, q1, q2, q3, q4, q5, q6, q7, q8))
        conn.commit()
        st.success("✅ تم حفظ الإجابات. سيتم تحليلها الآن.")
        st.image("ffd21d58-2e22-4afd-b179-3b1a07971f34.png", caption="نتيجة التقييم")

# Main app
st.set_page_config(page_title="منصة طمني", layout="centered")
st.title("🌿 منصة طمني - تقييم الصحة النفسية")

if 'user' not in st.session_state:
    page = st.sidebar.selectbox("اختر الصفحة", ["تسجيل الدخول", "تسجيل جديد"])
    if page == "تسجيل الدخول":
        login()
    else:
        signup()
else:
    questionnaire()
